In [249]:
# importing required libraries

import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

In [250]:
# target URL to scrap

base_url = 'https://nigeriapropertycentre.com/for-rent'

In [252]:
# send request to download the data

req = requests.get (base_url, headers=headers)

In [251]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}

In [253]:
req.status_code

200

In [254]:
req.text[1:200]

'!DOCTYPE html>\n<html lang="en">\n<head>\n<meta charset="utf-8">\n<meta name="apple-itunes-app" content="app-id=1125813120">\n<meta name="google-play-app" content="app-id=com.nigeriapropertycentre.app">\n<'

In [255]:
# parse the downloaded data

soup = BeautifulSoup(req.text,'html.parser')

In [256]:
soup.text[1:200]

"\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            Bugsnag.start({\n                apiKey: '2f9be6cbadc429acb199ec901c44bee7',\n                releaseStage: 'production'\n            })\n        \n\n        var app_short_name"

In [296]:
# find all title with specified class name

titles = soup.select('.content-title')

In [297]:
len(titles)

21

In [298]:
# fixing all the title data into a list

all_titles = []

for title in titles:
    all_titles.append(title.text[10:])

In [299]:
# fixing all the bedroom data into a list

all_bedrooms = []

for bedroom in titles:
    all_bedrooms.append(bedroom.text[:2])

In [300]:
# find all addresses with specified class name

addresses = soup.select('address')

In [301]:
len(addresses)

21

In [302]:
# fixing all the addresses data into a list

all_addresses = []

for address in addresses:
    all_addresses.append(address.text[0:15])
    

In [303]:
# find all descriptions with specified class name

descriptions = soup.select('.description > p')

In [304]:
len(descriptions)

21

In [305]:
# fixing all the descriptions data into a list

all_descriptions = []

for description in descriptions:
    all_descriptions.append(description.getText())


In [306]:
# find all prices with specified class name

prices = soup.select('.pull-sm-left > .price')

In [307]:
len(prices)

42

In [320]:
# fixing all the prices data into a list

all_prices = []

for i,price in enumerate(prices):
    if i % 2 != 0:
        all_prices.append(price.getText())
    

In [309]:
# find all facilities with specified class name

facilities = soup.select('.wp-block-footer >.aux-info')

In [321]:
len(facilities)

21

In [311]:
# fixing all the facilities data into a list

all_facilities = []

for facility in facilities:
    all_facilities.append(facility.getText())

In [322]:
# fixing all the bathrooms data into a list

all_toilets = []

for toilet in facilities:
    all_toilets.append(toilet.text[21:-29])

In [323]:
# fixing all the bathrooms data into a list

all_parking = []

for parking in facilities:
    all_parking.append(parking.text[30:-19])

In [324]:
#creating a function to be able to return all list generated

def all_data(url):
    req = requests.get (base_url, headers=headers)
    soup = BeautifulSoup(req.text,'html.parser')
    
    titles = soup.select('.content-title')
    all_titles = []
    for title in titles:
        all_titles.append(title.text[10:])    
    
    addresses = soup.select('address')
    all_addresses = []
    for address in addresses:
        all_addresses.append(address.text[0:15])
    
    descriptions = soup.select('.description > p')
    all_descriptions = []
    for description in descriptions:
        all_descriptions.append(description.getText())    
    
    facilities = soup.select('.wp-block-footer >.aux-info')
    all_facilities = []
    for facility in facilities:
        all_facilities.append(facility.getText())
    
    titles = soup.select('.content-title')
    all_bedrooms = []
    for bedroom in titles:
        all_bedrooms.append(bedroom.text[:2])
    
    facilities = soup.select('.wp-block-footer >.aux-info')
    all_toilets = []
    for toilet in facilities:
        all_toilets.append(toilet.text[21:-29])
    
    facilities = soup.select('.wp-block-footer >.aux-info')
    all_parking = []
    for parking in facilities:
        all_parking.append(parking.text[30:-19])
    
    prices = soup.select('.pull-sm-left > .price')
    all_prices = []
    for i,price in enumerate(prices):
        if i % 2 != 0:
            all_prices.append(price.getText())
            
    return (all_titles, all_addresses, all_descriptions, all_facilities, all_bedrooms, all_toilets, all_parking, all_prices)   

In [325]:
#this code helps to crawl all the pages on the website

urls = [base_url]
for i in range(1,50):
    url = base_url + '?page=' + str(i)
    urls.append(url)

In [326]:
urls[49]

'https://nigeriapropertycentre.com/for-rent?page=49'

In [327]:
master_titles = []
master_addresses = []
master_bedrooms = []
master_toilets = []
master_parking = []
master_prices = []

for url in urls:
    all_titles, all_addresses, all_descriptions, all_facilities, all_bedrooms, all_toilets, all_parking, all_prices = all_data(url)
    
    master_titles += all_titles
    master_addresses += all_addresses
    master_bedrooms += all_bedrooms
    master_toilets += all_toilets
    master_parking += all_parking
    master_prices += all_prices
    
    time.sleep(20)

In [328]:
# create a data frame from the list of dictionaries

properties = pd.DataFrame ({"property": master_titles,
                            "address": master_addresses,
                            "bedroom": master_bedrooms,
                            "toilet": master_toilets,
                            "parking_space": master_parking,
                            "prices": master_prices
                           })

In [329]:
len(master_toilets)

1050

In [330]:
properties.to_csv("property_for_rent.csv")

In [331]:
house_for_rent = pd.read_csv('property_for_rent.csv')

In [332]:
house_for_rent.head()

,Unnamed: 0,property,address,bedroom,toilet,parking_space,prices
0,0,semi-detached duplex for rent,"Ikota, Lekki,",4,5,3,"3,500,000"
1,1,flat for rent,Divine Homes,3,4,2,"1,500,000"
2,2,terraced duplex for rent,Maitama Distr,4,5,3,"10,000,000"
3,3,flat for rent,Off Palace Ro,3,4,2,"4,500,000"
4,4,terraced duplex for rent,Osborne Fores,4,NaN,NaN,"10,000,000"
